In [1]:
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from tqdm import tqdm 
from sklearn.preprocessing import StandardScaler
import sr_processing as sr

In [2]:
def psth_cal(trial_arrays):
    stacked_trial_rates = np.stack(trial_arrays, axis=0)
    per_neuron_psth = np.mean(stacked_trial_rates, axis=0)
    return per_neuron_psth

In [3]:
stims_eps = 'RFMapping_2'
epochs_path = f'/scratch/{stims_eps}/'
fr_path = f'{epochs_path}/firing_rate2/'
trial_path = os.path.join(epochs_path,'trial_info.csv')
df_trials = pd.read_csv(trial_path,index_col='Unnamed: 0')

In [4]:
index_map = sr.get_index_maps(df_trials)

In [7]:
results_dic ={}
pbar = tqdm(total = len(index_map.keys()),leave=True, position=0)

for inds in index_map.keys():
    trials2avg = index_map[inds]
    data_set = {d_ind:sr.get_spikerate_data(f'{fr_path}trial_array_{d_ind}.pkl') for d_ind in trials2avg}
    if len(set([d['bin_centers'].shape[0] for d in data_set.values()])) != 1:
        data_set = sr.trim_binsz_ts(data_set)
    results_dic[inds] = psth_cal([trial['s_counts'] for trial in data_set.values()])
    pbar.update(1)
filename = f'{epochs_path}firing_rates.pkl'
pickle.dump(results_dic, open(filename, 'wb'))

 98%|█████████▊| 190/193 [00:04<00:00, 45.71it/s]